# Практическая работа №2: Обработка выборочных данных. Нахождение точечных оценок параметров распределения

Выполнили студенты гр. 0383 Желнин Максим, Рудакова Юлия, Сергевнин Дмитрий. Вариант №12


## Цель работы
Получение практических навыков нахождения точечных статистических оценок параметров распределения.

## Основные теоретические положения

**Выборка** - подмножество элементов, взятых из генеральной совокупности, которое используется для получения информации о данной совокупности.

**Генеральная совокупность** - полный набор всех объектов или событий, о которых вы хотите сделать выводы.

**Статистический ряд** - последовательность элементов выборки, расположенных в порядке их получения.

**Ранжированный ряд** - это упорядоченный список значений из выборки или генеральной совокупности по возрастанию или убыванию.

**Вариационный ряд** - это ряд, в котором данные представлены в виде уникальных значений, отсортированных по возрастанию или убыванию.

**Интервальный ряд** – это представление данных в виде интервалов, в которых группируются значения, чтобы облегчить их анализ.

Количество интервалов $k$ зависит от объема выборки $N$ и может быть вычислено по формуле Стерджесса:
$$k = 1+3.31*lgN$$
Полученное значение округляется до целого. Рекомендуется выбирать нечетное количество интервалов.

**Математическое ожидание** дискретной случайной величины - сумма произведений ее возможных значений на соответствующие им вероятности:

$$M(X) = \frac {1} {N} \sum_{i=1}^n x_in_i$$

**Дисперсия** - математическое ожидание квадрата отклонения от математического ожидания случайной величины. В свою очередь **среднеквадратичное отклонение** - это квадрат из дисперсии.

$$D(X) = M(X-M(X))^2$$

**Выборочная дисперсия и исправленная выборочная дисперсия**:

$$D_B = \frac {1} {N} \sum_{i=1}^k (x_i - x)^2 $$

$$s^2 = \frac {N} {N - 1} D_B$$


**Центральный момент порядка k**:

$$M(X-M(X))^k = M_k$$

**Асимметрия** опредеяется центральным эмпирическим моментом третьего порядка и исправленной выборочной дисперсий:

$$A_s =  \frac {M_3} {s^3}$$

**Эксцесс**:

$$E = \frac {M_4} {s^4} - 3$$

**Мода** – наиболее вероятное значение случайной величины.

$$M_o = x_o + \frac {M_2 - M_1} {(M_2-M_1) + (M_2-M_3)} h$$


## Постановка задачи

Для заданных выборочных данных вычислить с использованием метода моментов и условных вариант точечные статистические оценки математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса, моды, медианы и коэффициента вариации исследуемой случайной величины. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы


### 1.
Для середин интервального ряда, полученного в практической работе №1, вычислить условные варианты. Заполнить табл. 1 (в последней строке
Σ необходимо заполнить суммы столбцов; ячейки отмеченные прочерком заполнять не надо). Провести контроль вычислений.

Сначала повторим шаги из 1 лабораторной для получения интервалов:

In [1]:
import pandas as pd
import numpy as np
import io
import zipfile
from google.colab import files
import math
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt

In [2]:
uploaded = files.upload()

Saving sample.zip to sample.zip


In [3]:
# Извлекаем файл sample.csv из архива
with zipfile.ZipFile(io.BytesIO(uploaded['sample.zip']), 'r') as zip_ref:
    zip_ref.extractall()

df = pd.read_csv('sample.csv', skiprows=3) # двумерная генеральная совокупность

In [4]:
N = 111
sample = df.sample(n=N, random_state=1)  # random_state для воспроизводимости (дает возможность воспроизводить те же самые случайные результаты при каждом запуске кода)

In [5]:
ranked_nu = sample.sort_values(by='nu').reset_index(drop=True)['nu']
ranked_e = sample.sort_values(by='E').reset_index(drop=True)['E']

In [6]:
# Преобразование ранжированного ряда в вариационный ряд для 'nu'
variation_nu = ranked_nu.value_counts().reset_index()
variation_nu.columns = ['значение', 'частота']
variation_nu_sort = variation_nu.sort_values(by='значение').reset_index(drop=True)
variation_nu_sort.drop(columns=['частота'], inplace=True)


# Преобразование ранжированного ряда в вариационный ряд для 'E'
variation_e = ranked_e.value_counts().reset_index()
variation_e.columns = ['значение', 'частота']
variation_e_sort = variation_e.sort_values(by='значение').reset_index(drop=True)
variation_e_sort.drop(columns=['частота'], inplace=True)

In [7]:
def create_interval_series(variation_series):
    # Определение интервалов
    num_bins = round(1 + 3.31 * math.log10(N))
    if num_bins % 2 == 0:
      num_bins += 1

    bin_width = (variation_series['значение'].max() - variation_series['значение'].min()) / num_bins
    bins = [variation_series['значение'].min() + i * bin_width for i in range(num_bins)]
    bins.append(variation_series['значение'].max())

    # Подсчет частоты попадания в интервалы
    frequency, _ = np.histogram(variation_series['значение'], bins=bins)

    # Определение границ и середин интервалов
    interval_bounds = [(bins[i], bins[i+1]) for i in range(num_bins)]
    interval_midpoints = [(interval[0] + interval[1]) / 2 for interval in interval_bounds]

    interval_series = pd.DataFrame({
        'Границы интервалов': interval_bounds,
        'Середины интервалов': interval_midpoints,
        'Частота попадания в интервал': frequency,
        'Относительная частота': frequency / len(variation_series)
    })

    return interval_series


interval_nu_series = create_interval_series(variation_nu_sort)

interval_e_series = create_interval_series(variation_e_sort)
print(interval_e_series['Относительная частота'])

0    0.028302
1    0.094340
2    0.122642
3    0.122642
4    0.179245
5    0.169811
6    0.160377
7    0.066038
8    0.056604
Name: Относительная частота, dtype: float64


Распределим значения в массивы по интервалам:

In [8]:
def get_buckets(intervals, series):
    buckets = [[] for i in range(len(intervals) - 1)]
    for i in range(len(series) - 1):
      for j in range(len(intervals) - 1):
        if series[i] > intervals[j][0] and series[i] <= intervals[j][1]:
          buckets[j].append(series[i])
    return buckets

Для NU:

In [9]:
mid_nu_values = interval_nu_series["Середины интервалов"]

buckets_nu = get_buckets(interval_nu_series["Границы интервалов"], variation_nu["значение"])
C_nu = mid_nu_values[max([(len(bucket), i) for i, bucket in enumerate(buckets_nu)])[1]]
h_nu = int((mid_nu_values[len(mid_nu_values) - 1] - mid_nu_values[0]) /(len(mid_nu_values)-1))
ui_arr_nu = [round((xi - C_nu) / h_nu) for xi in mid_nu_values]
nu_freq_arr = interval_nu_series["Частота попадания в интервал"]
len_intervals_nu = len(ui_arr_nu)

data_nu = {'xi': mid_nu_values,
        'ni': nu_freq_arr,
        'ui':  ui_arr_nu,
        'ui*n': [nu_freq_arr[i] * ui_arr_nu[i] for i in range(len_intervals_nu)],
        'ui^2 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 2) for i in range(len_intervals_nu)],
        'ui^3 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 3) for i in range(len_intervals_nu)],
        'ui^4 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 4) for i in range(len_intervals_nu)],
        '(ui + 1)^4 * n': [nu_freq_arr[i] * ((ui_arr_nu[i] + 1) ** 4) for i in range(len_intervals_nu)]}

table1_nu = pd.DataFrame(data_nu)
table1_nu.loc['Sum'] = table1_nu.sum()
print("Таблица 1:")
print(table1_nu)

Таблица 1:
              xi    ni   ui  ui*n  ui^2 * n  ui^3 * n  ui^4 * n  \
0     335.055556   3.0 -5.0 -15.0      75.0    -375.0    1875.0   
1     365.166667   8.0 -4.0 -32.0     128.0    -512.0    2048.0   
2     395.277778  13.0 -3.0 -39.0     117.0    -351.0    1053.0   
3     425.388889  12.0 -2.0 -24.0      48.0     -96.0     192.0   
4     455.500000  13.0 -1.0 -13.0      13.0     -13.0      13.0   
5     485.611111  15.0  0.0   0.0       0.0       0.0       0.0   
6     515.722222  11.0  1.0  11.0      11.0      11.0      11.0   
7     545.833333   8.0  2.0  16.0      32.0      64.0     128.0   
8     575.944444   3.0  3.0   9.0      27.0      81.0     243.0   
Sum  4099.500000  86.0 -9.0 -87.0     451.0   -1191.0    5563.0   

     (ui + 1)^4 * n  
0             768.0  
1             648.0  
2             208.0  
3              12.0  
4               0.0  
5              15.0  
6             176.0  
7             648.0  
8             768.0  
Sum          3243.0  


Успешная проверка по формуле $5563 - 4*1191 + 6*451 - 4*87 + 86 = 3243$


Для E:

In [10]:
mid_e_values = interval_e_series["Середины интервалов"]

buckets_e = get_buckets(interval_e_series["Границы интервалов"], variation_e["значение"])
C_e = mid_e_values[max([(len(bucket), i) for i, bucket in enumerate(buckets_e)])[1]]
h_e = int((mid_e_values[len(mid_e_values) - 1] - mid_e_values[0]) /(len(mid_e_values)-1))
ui_arr_e = [(xi - C_e) / h_e for xi in mid_e_values]
e_freq_arr = interval_e_series["Частота попадания в интервал"]
len_intervals_e = len(ui_arr_e)

data_e = {'xi': mid_e_values,
        'ni': e_freq_arr,
        'ui':  ui_arr_e,
        'ui*n': [e_freq_arr[i] * ui_arr_e[i] for i in range(len_intervals_e)],
        'ui^2 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 2) for i in range(len_intervals_e)],
        'ui^3 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 3) for i in range(len_intervals_e)],
        'ui^4 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 4) for i in range(len_intervals_e)],
        '(ui + 1)^4 * n': [e_freq_arr[i] * ((ui_arr_e[i] + 1) ** 4) for i in range(len_intervals_e)]}

table1_e = pd.DataFrame(data_e)
table1_e.loc['Sum'] = table1_e.sum()
print("Таблица 1:")
print(table1_e)

Таблица 1:
              xi     ni            ui       ui*n    ui^2 * n    ui^3 * n  \
0      78.622222    3.0 -4.014815e+00 -12.044444   48.356214 -194.141244   
1      90.666667   10.0 -3.011111e+00 -30.111111   90.667901 -273.011125   
2     102.711111   13.0 -2.007407e+00 -26.096296   52.385898 -105.159841   
3     114.755556   13.0 -1.003704e+00 -13.048148   13.096475  -13.144980   
4     126.800000   19.0  0.000000e+00   0.000000    0.000000    0.000000   
5     138.844444   18.0  1.003704e+00  18.066667   18.133580   18.200742   
6     150.888889   17.0  2.007407e+00  34.125926   68.504636  137.516715   
7     162.933333    7.0  3.011111e+00  21.077778   63.467531  191.107787   
8     174.977778    6.0  4.014815e+00  24.088889   96.712428  388.282489   
Sum  1141.200000  106.0 -2.664535e-15  16.059259  451.324664  149.650542   

        ui^4 * n  (ui + 1)^4 * n  
0     779.441144    2.478357e+02  
1     822.066831    1.635853e+02  
2     211.098643    1.338949e+01  
3      13.19

Проверка: $4192 + 4 * 148 + 6*448 + 4*16 + 106 = 7642$ — сошлось

###2.
Вычислить условные эмпирические моменты νi* через условные варианты. С помощью условных эмпирических моментов вычислить центральные эмпирические моменты μi\*. Полученные результаты занести в табл. 2.

In [11]:
nu_N = sum(data_nu["ni"])
vi_nu_arr = [sum(data_nu["ui*n"]) / nu_N, sum(data_nu["ui^2 * n"]) / nu_N, sum(data_nu["ui^3 * n"]) / nu_N, sum(data_nu["ui^4 * n"]) / nu_N]
mi_nu_arr = [0, (vi_nu_arr[1] - vi_nu_arr[0] ** 2) * (h_nu ** 2), (vi_nu_arr[2] - 3* vi_nu_arr[1] * vi_nu_arr[0]  + 2 * (vi_nu_arr[0]** 3)) * (h_nu ** 3), (vi_nu_arr[3] - 4 * vi_nu_arr[2] * vi_nu_arr[0] + 6 * vi_nu_arr[1] * (vi_nu_arr[0] ** 2)  - 3 * (vi_nu_arr[0]** 4)) * (h_nu ** 4)]
table2_data_nu = {
        'vi': vi_nu_arr,
        'mi':  mi_nu_arr}

table2_nu = pd.DataFrame(table2_data_nu)
print("Таблица 2:")
print(table2_nu)

Таблица 2:
          vi            mi
0  -1.011628  0.000000e+00
1   5.244186  3.798716e+03
2 -13.848837 -1.059529e+02
3  64.686047  3.054161e+07


In [12]:
vi_e_arr = [sum(data_e["ui*n"]) / N, sum(data_e["ui^2 * n"]) / N, sum(data_e["ui^3 * n"]) / N, sum(data_e["ui^4 * n"]) / N]
mi_e_arr = [0, (vi_e_arr[1] - vi_e_arr[0] ** 2) * (h_e ** 2), (vi_e_arr[2] - 3* vi_e_arr[1] * vi_e_arr[0]  + 2 * (vi_e_arr[0]** 3)) * (h_e ** 3), (vi_e_arr[3] - 4 * vi_e_arr[2] * vi_e_arr[0] + 6 * vi_e_arr[1] * (vi_e_arr[0] ** 2)  - 3 * (vi_e_arr[0]** 4)) * (h_e ** 4)]
table2_data_e = {
        'vi': vi_e_arr,
        'mi':  mi_e_arr}

table2_e = pd.DataFrame(table2_data_e)
print("Таблица 2:")
print(table2_e)

Таблица 2:
          vi             mi
0   0.144678       0.000000
1   4.065988     582.488098
2   1.348203    -709.373983
3  38.328375  789160.064737


###3.
Вычислить выборочные среднее $x_{в ср}$ и дисперсию $σ^{2}_{в}$
 с помощью стандартной формулы и с помощью условных вариант. Убедиться, что результаты совпадают.

 Далее проведём вычисления только для e

In [13]:
x_v = vi_e_arr[0] * h_e + C_e
x_v2 = sum(ranked_e)/N

sigma = mi_e_arr[1]
sigma2 = sum([(ranked_e[i] - x_v2) ** 2 for i in range(N)]) / N

print(x_v, x_v2)
print(sigma, sigma2)

128.53613613613612 128.63243243243238
582.4880980279578 587.9000292184074


Результаты почти совпадают, однако дисперсия имеет существенное отличие, которое, в теории, может быть результатом погрешности округления

###4.
Вычислить исправленную выборочную дисперсию $S^{2}$ и исправленное СКО S. Сравнить данные оценки с смещёнными оценками дисперсии. Сделать выводы.

In [14]:
S_square = sigma2 * N / (N-1)
S = np.sqrt(S_square)
print(S_square, S)

593.2445749385748 24.35661255056981


Исправленная выборочная дисперсия отличается от значения выборочной
дисперсии, так как последняя является смещенной оценкой, несколько
завышающие значения параметров распределения.

###5.
Найти статистическую оценку коэффициентов асимметрии $a^{*}_{s}$ и эксцесса $ε^{*}_{s}$. Сделать выводы.

In [15]:
a_s = mi_e_arr[2] / (S ** 3)
e_s = mi_e_arr[3] / (S ** 4) - 3
print(a_s, e_s)

-0.049093567588123006 -0.7576802147789534


1. Коэффициент асимметрии (-0.049093567588123006) близок к нулю, что говорит о том, что распределение данных почти симметрично относительно их среднего значения. Такое значение коэффициента асимметрии указывает на отсутствие явной асимметрии в данных.

2. Коэффициент эксцесса (-0.7576802147789534) отрицателен, что указывает на то, что пик распределения данных более плоский (более пологий) по сравнению с нормальным распределением. Такое значение коэффициента эксцесса говорит о том, что данные имеют менее острый пик, чем у нормального распределения.

###6.
Для интервального ряда вычислить моду $M^{*}_{o}$, медиану $M^{*}_{e}$ и коэффициент вариации $V^{*}$ заданного распределения. Сделать выводы.

Самая высокая относительная частота у 5 интервала 0.179245

In [16]:
moda = data_e["xi"][3] + h_e * (interval_e_series['Относительная частота'][4]-interval_e_series['Относительная частота'][3])/(interval_e_series['Относительная частота'][4]-interval_e_series['Относительная частота'][3] + interval_e_series['Относительная частота'][4]-interval_e_series['Относительная частота'][5])
print(moda)

125.04126984126984


In [28]:
left_border = interval_e_series["Границы интервалов"][4][0]
mediana = left_border + (interval_e_series["Границы интервалов"][4][1] - left_border)*(sum(e_freq_arr)/2 -  sum(interval_e_series["Частота попадания в интервал"][0:4]))/ e_freq_arr[4]
print(mediana)

129.65263157894736


Медиана находится в пятом интервале, так как его накопленная частота больше половины и равняется 129.65263157894736

In [19]:
print(x_v2 / sigma2 * 100)

21.879984017596446


Коэффициент вариации = 21% => данные имеют относительно высокий уровень вариабельности или разброса относительно своего среднего значения

##Вывод


В результате выполнения данной практической работы были получены
практические навыки нахождения точечных оценок параметров распределения с
помощью метода моментов. В ходе выполнения работы были найдены точечные
оценки математического ожидания, дисперсии (смещенная и несмещенная),
среднеквадратического отклонения, коэффициентов асимметрии и эксцесса, а
также проанализированы полученные значения.